In [ ]:
from dotenv import load_dotenv

load_dotenv()
from falkordb_gemini_kg.classes.model_config import (
    KnowledgeGraphModelConfig,
    KnowledgeGraphModelStepConfig,
)
from falkordb_gemini_kg import KnowledgeGraph, Ontology
from falkordb_gemini_kg.classes.source import HTML
import vertexai
import os
from random import shuffle
import json


vertexai.init(project=os.getenv("PROJECT_ID"), location=os.getenv("REGION"))

In [ ]:
src_file="../data/rottentomatoes.txt"

source_urls = []
with open(src_file, "r", encoding="utf-8") as file:
    source_urls = file.readlines()

shuffle(source_urls)

sources = [HTML(url) for url in source_urls]

In [ ]:
ontology = Ontology.from_sources(
    sources=sources[: round(len(sources) * 0.2)],
    boundaries=input("Enter the boundaries for the ontology creation:\n"),
    model_config=KnowledgeGraphModelStepConfig(model="gemini-1.5-flash-001"),
)

ontology_file = "ontology.json"

# Save to json file
with open(ontology_file, "w", encoding="utf-8") as file:
    file.write(json.dumps(ontology.to_json(), indent=2))

In [ ]:
# Read from json file
ontology_file = "ontology.json"
with open(ontology_file, "r", encoding="utf-8") as file:
    ontology = Ontology.from_json(json.loads(file.read()))

kg = KnowledgeGraph(
    name="movies",
    model_config=KnowledgeGraphModelConfig.from_dict(d={"model": "gemini-1.5-flash-001"}),
    ontology=ontology,
)

In [ ]:
kg.process_sources(sources)

In [ ]:
kg.ask("How are Keanu Reeves and Carrie-Anne Moss related?")

In [ ]:
chat = kg.chat_session()

chat.send_message("Who is the director of the movie The Matrix?")
chat.send_message("And how are they related with Keanu Reeves?")